<a href="https://colab.research.google.com/github/djmary-k/Py16DS_HW03/blob/main/Hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 03, Module 03, Data Science.
Дане домашнє завдання буде повністю пов'язане з лінійною регресією та її реалізацією.

Розіб'ємо наше домашнє завдання на декілька частин:
* напишіть функцію гіпотези лінійної регресії у векторному вигляді;
* створіть функцію для обчислення функції втрат у векторному вигляді;
* реалізуйте один крок градієнтного спуску;

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
def cost(h, y):
  return np.mean((h-y)**2)/2

class CustomLinearRegression:
  def __init__(self, lr: float = 0.01, thr=0.000000001, n_epochs = 1000):
    # lr - learning rate(коефіцієнт навчання, який відповідає за те, наскільки великий крок буде робити алгоритм на кожній ітерації)
    # thr - критерій зупинки навчання або трешхолд
    # n_epoch - кількість епох\ кроки градієнтного спуску (кількість ітерацій \ num_iter)
    self.lr = lr # alfa
    self.thr = thr # epsilon
    self.n_epochs = n_epochs # num_iter
    # self.w = np.array([np.random.uniform(-5,5)])
    self.w = np.array([np.random.uniform(-5,5), np.random.uniform(-5,5), np.random.uniform(-5,5)])

  def predict(self, X): # гіпотетична модель h(x)
    return self.w @ X.T

  def update_weights(self, X, y): #реалізація одиного кроку градієнтного спуску grad_step
    m = len(y)
    h = self.predict(X)
    self.w = self.w - self.lr / m * X.T @ (h-y)

  def fit(self, X, y): # цикл із кроків градієнтного спуску grade_descent
    last_cost = 10000000 #рандомне величезне число
    for i in range(self.n_epochs):
      self.update_weights(X, y)
      new_cost = cost(self.predict(X), y)

      if i % (self.n_epochs//1000) == 0:
        print(f'cost function = {last_cost}')

      if abs(last_cost - new_cost) < self.thr: # зупинка навчання
        break

      last_cost = new_cost



* **знайдіть найкращі параметри $w$ для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень**

In [6]:
data_housing = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Housing.csv')
data_housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
# data_housing.plot(x='area', y='price', kind='scatter')

In [7]:
# Split the data into training/testing sets
X_train = data_housing.loc[:500, ['area', 'bathrooms', 'bedrooms']]
X_test = data_housing.loc[501:, ['area', 'bathrooms', 'bedrooms']]

# Split the targets into training/testing sets
y_train = data_housing.loc[:500, 'price']
y_test = data_housing.loc[501:, 'price']



In [8]:
# normalization data: 'X' and 'y'

def normalization(data):
  mean_data = np.mean(data)
  value_range = np.max(data) - np.min(data)
  result = []

  for x in data:
    norm_x = (x - mean_data) / value_range
    result.append(norm_x)

  return result

In [ ]:
# # y - Vector [prices]
# # X - Matrix [area]

# norm_y = np.array(normalization(data_housing.price))
# norm_x = pd.DataFrame()
# norm_x['area'] = normalization(data_housing.area)

In [9]:
# y - Vector [prices]
# X - Matrix [area, bathrooms, bedrooms]

norm_y = np.array(normalization(y_train))
norm_x = pd.DataFrame()
norm_x['area'] = normalization(X_train.area)
norm_x['bathrooms'] = normalization(X_train.bathrooms)
norm_x['bedrooms'] = normalization(X_train.bedrooms)

In [ ]:
cust_regression = CustomLinearRegression(lr=0.1, thr=0.00000000000001)
# cust_regression = CustomLinearRegression(lr=0.1, thr=0.000001)
cust_regression.fit(norm_x, norm_y)

In [24]:
cust_regression.w

area         0.223417
bathrooms    0.502759
bedrooms     0.357496
dtype: float64

* **знайдіть ці ж параметри за допомогою аналітичного рішення**

In [11]:
# 1
def analytical_solution(X, y):
    return np.linalg.pinv(X) @ y

In [28]:
w_analytical_1 = analytical_solution(norm_x, norm_y)
w_analytical_1

array([0.46607398, 0.37812754, 0.17297334])

In [43]:
# 2
w_analytical_2 = np.linalg.inv(norm_x.T @ norm_x) @ norm_x.T @ norm_y
w_analytical_2

0    0.466074
1    0.378128
2    0.172973
dtype: float64

* **реалізація за допомогою бібліотеки sklearn**

In [31]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression().fit(norm_x, norm_y)
# коефіцієнти
print(regressor.coef_)
# зсув
print(regressor.intercept_)

[0.46607398 0.37812754 0.17297334]
-2.95177287828457e-17


* **порівняйте отримані результати**

Отже, параметри $w$ для даного датасету:
1. Отримані за допомогою кастомної реалізації

**area         0.223417**

**bathrooms    0.502759**

**bedrooms     0.357496**

2. Отримані за допомогою аналітичних рішеннь:
- **[0.46607398, 0.37812754, 0.17297334]**
- **0    0.466074** / **1    0.378128** / **2    0.172973**

3. Отримані за допомогою бібліотеки scikit learn:

**[0.47714269 0.36001286 0.17611257]**


*Щоб визначити який результат кращий, спрогнозуємо ціни для датасету **X_test** за допомогою отриманих параметрів $w$ і порівняємо їх з цільовим датасетом **y_test**.*

In [70]:
# Make predictions using the testing set
# 1 solution
cust_solution = cust_regression.predict(X_test)
# 2 solution
analytical_solution_1 = X_test @ w_analytical_1
analytical_solution_2 = X_test @ np.array(w_analytical_2)
# 3 solution
sklearn_solution = regressor.predict(X_test)

In [69]:
result = pd.DataFrame(
    {
        'Target prices': y_test,
        'Prediction prices (custom)': cust_solution,
        'Prediction prices (analytical 1)': analytical_solution_1,
        'Prediction prices (analytical 2)': analytical_solution_2,
        'Prediction prices (sklearn)': sklearn_solution
    }
)
result

,Target prices,Prediction prices (custom),Prediction prices (analytical 1),Prediction prices (analytical 2),Prediction prices (sklearn)
501,2660000,544.477515,1133.456823,1133.456823,1133.456823
502,2660000,778.707419,1622.661530,1622.661530,1622.661530
503,2660000,895.241532,1865.192974,1865.192974,1865.192974
504,2653000,712.799530,1485.169706,1485.169706,1485.169706
505,2653000,895.241532,1865.192974,1865.192974,1865.192974
506,2604000,651.359973,1356.999361,1356.999361,1356.999361
507,2590000,805.517408,1678.590408,1678.590408,1678.590408
508,2590000,984.250665,2051.449593,2051.449593,2051.449593
509,2590000,806.020167,1678.968535,1678.968535,1678.968535
510,2520000,645.014972,1343.190115,1343.190115,1343.190115
